In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import cProfile

import numpy as np

import kraft

In [3]:
for power in range(9):

    n = 2 ** power

    print(
        "{} ({} random)".format(
            kraft.compute_vector_entropy(np.random.random_sample(size=n)), n
        )
    )

    for fill_value in (1, 2, 8):

        print(
            "{} ({} {})".format(
                kraft.compute_vector_entropy(np.full(n, fill_value)), n, fill_value
            )
        )

-0.0 (1 random)
-0.0 (1 1)
-0.0 (1 2)
-0.0 (1 8)
0.5141716635915232 (2 random)
0.6931471805599453 (2 1)
0.6931471805599453 (2 2)
0.6931471805599453 (2 8)
1.2630401892005458 (4 random)
1.3862943611198906 (4 1)
1.3862943611198906 (4 2)
1.3862943611198906 (4 8)
1.896025820760651 (8 random)
2.0794415416798357 (8 1)
2.0794415416798357 (8 2)
2.0794415416798357 (8 8)
2.553295071098641 (16 random)
2.772588722239781 (16 1)
2.772588722239781 (16 2)
2.772588722239781 (16 8)
3.2483780500679975 (32 random)
3.4657359027997265 (32 1)
3.4657359027997265 (32 2)
3.4657359027997265 (32 8)
3.944483890502889 (64 random)
4.1588830833596715 (64 1)
4.1588830833596715 (64 2)
4.1588830833596715 (64 8)
4.665419594871183 (128 random)
4.852030263919617 (128 1)
4.852030263919617 (128 2)
4.852030263919617 (128 8)
5.367323300730376 (256 random)
5.545177444479562 (256 1)
5.545177444479562 (256 2)
5.545177444479562 (256 8)


In [4]:
for vector_0, vector_1 in (
    (np.asarray((0,) * 8),) * 2,
    (np.asarray((1,) * 8),) * 2,
    (np.asarray(range(8)), np.asarray(range(8))[::-1]),
    (np.asarray((0, 1, 2, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 2, 1, 0)),) * 2,
    (np.asarray((8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 6, 7, 8)),) * 2,
    (np.asarray((0, 1) * 8), np.asarray((0, 1) * 8)),
    (np.asarray((0, 1) * 8), np.asarray((1, 0) * 8)),
):

    print(vector_0)

    print(vector_1)

    print(kraft.compute_information_coefficient_between_2_vectors(vector_0, vector_1))

    print(kraft.compute_information_distance_between_2_vectors(vector_0, vector_1))

    print()

[0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0]
nan
nan

[1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1]
nan
nan

[0 1 2 3 4 5 6 7]
[7 6 5 4 3 2 1 0]
-0.9375713920264418
0.9687856960132208

[0 1 2 3 4 5 6 7 8 7 6 5 4 3 2 1 0]
[0 1 2 3 4 5 6 7 8 7 6 5 4 3 2 1 0]
0.95087157525268
0.02456421237366002

[8 7 6 5 4 3 2 1 0 1 2 3 4 5 6 7 8]
[8 7 6 5 4 3 2 1 0 1 2 3 4 5 6 7 8]
0.9508715752526798
0.024564212373660077

[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
0.8660254037843874
0.06698729810780629

[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]
-0.8660254037843874
0.9330127018921937



In [5]:
for power in range(9):

    n = 2 ** power

    vector_0 = np.random.normal(size=n)

    vector_1 = np.random.normal(size=n)

    print(
        "{} ({} random)".format(
            kraft.compute_information_coefficient_between_2_vectors(vector_0, vector_1),
            n,
        )
    )

nan (1 random)
-0.8649528464501417 (2 random)
-0.709939991282416 (4 random)
-0.32954532664321634 (8 random)
0.5121486251957619 (16 random)
-0.38371831292063346 (32 random)
-0.26907422549083154 (64 random)
-0.2535807832513904 (128 random)
0.23238475956765514 (256 random)


In [6]:
n = 2

vector_0 = np.asarray((0,) * n + (1,) * n + (2,) * n)

for factor in (1, 2, 1 / 2, 10, 1 / 10):

    vector_1 = vector_0 * factor

    print(vector_0)

    print(vector_1)

    print(kraft.compute_information_coefficient_between_2_vectors(vector_0, vector_1))

    print()

[0 0 1 1 2 2]
[0 0 1 1 2 2]
0.9355697716010843

[0 0 1 1 2 2]
[0 0 2 2 4 4]
0.9355697716010843

[0 0 1 1 2 2]
[0.  0.  0.5 0.5 1.  1. ]
0.9355697716010843

[0 0 1 1 2 2]
[ 0  0 10 10 20 20]
0.9355697716010843

[0 0 1 1 2 2]
[0.  0.  0.1 0.1 0.2 0.2]
0.9355697716010843



In [7]:
n = 100

vector_0 = np.random.random_sample(size=n)

vector_1 = np.random.random_sample(size=n)

In [8]:
kraft.compute_information_coefficient_between_2_vectors(vector_0, vector_1)

0.25547097691480564

In [9]:
%%timeit

kraft.compute_information_coefficient_between_2_vectors(
    vector_0,
    vector_1,
)

4.01 ms ± 267 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
cProfile.run(
    "kraft.compute_information_coefficient_between_2_vectors(vector_0, vector_1)",
    sort="cumtime",
)

         2730 function calls (2587 primitive calls) in 0.005 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.005    0.005 {built-in method builtins.exec}
        1    0.000    0.000    0.005    0.005 <string>:1(<module>)
        1    0.000    0.000    0.005    0.005 compute_information_coefficient_between_2_vectors.py:14(compute_information_coefficient_between_2_vectors)
        1    0.000    0.000    0.004    0.004 compute_element_x_dimension_joint_probability.py:12(compute_element_x_dimension_joint_probability)
        1    0.000    0.000    0.004    0.004 estimate_element_x_dimension_kernel_density.py:14(estimate_element_x_dimension_kernel_density)
        1    0.000    0.000    0.004    0.004 FFTKDE.py:105(evaluate)
  247/104    0.000    0.000    0.002    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
       13    0.000    0.000    0.002    0.000 kernel_f